# Задание 4

## 1. Chainer (0.4 балла)

Требуется реализовать функцию chainer, которая принимает на вход любое количество итерируемых объектов (в т.ч. контейнеров, которые тоже могут содержать итерируемые объекты) и возвращает список из всех элементов всех переданных контейнеров.

## Вариант с for:

In [114]:
def check_is_one_str_symb(str_element: str) -> bool:
    return len(str_element) > 1 or not isinstance(str_element, str)

def chainer_for(*args):
    res = []

    def inner_itter(el):
        nonlocal res

        if hasattr(el, '__iter__') and len(el) > 0 and \
                check_is_one_str_symb(el):
            for i in el:
                inner_itter(i)
        else:
            res.append(el)

    inner_itter(args)
    return res

## Без for:

In [115]:
def check_is_one_str_symb(str_element: str) -> bool:
    return len(str_element) > 1 or not isinstance(str_element, str)


def chainer(*args):
    res = []

    def _iter(item_):
        nonlocal res

        def _inner_iter(inner_item_):
            if hasattr(inner_item_, '__iter__') and len(inner_item_) > 0 \
                    and check_is_one_str_symb(inner_item_):
                _iter(inner_item_)
            else:
                res.append(inner_item_)

        list(map(_inner_iter, item_))

    list(map(_iter, args))
    return res

Пример работы функции:

In [116]:
print(chainer([1, 2, 3], "why", ["i need", ["do that", ["?"]]]))

[1, 2, 3, 'w', 'h', 'y', 'i', ' ', 'n', 'e', 'e', 'd', 'd', 'o', ' ', 't', 'h', 'a', 't', '?']


При этом запрещается использовать в коде конструкции циклов while и for.

## 2. Dataflow (0.6 балла)

В данном задании требуется написать обработчик генераторов, вычисляющих факториал. Для каждого генератора должна быть функция декоратор, которая инициализирует генератор.

In [89]:
from functools import wraps,reduce

def coroutine(func):
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        generator = func(*args, **kwargs)
        return generator
    
    return wrapper

Функция генератор факториалов принимает на вход n - максимальный факториал, который может вычислить ($n!$).

In [90]:
@coroutine
def fact_generator(n):
    factor = 1
    res = 1

    while factor <= n:
        upper_limit = yield res 
        if upper_limit is not None and upper_limit > factor:
            res *= reduce(lambda a, b: a * b, range(factor + 1, upper_limit + 1))     
            factor = upper_limit
        elif upper_limit is not None:
            raise StopIteration("Attempt to rollback generator state.")
        else:
            factor += 1
            res *= factor

In [92]:
f10 = fact_generator(10)
while True:
    print(next(f10))

1
2
6
24
120
720
5040
40320
362880
3628800


StopIteration: 

In [75]:
f7 = fact_generator(7)

for i in range(7):
    print(next(f7))

    if i == 2:
        try:
            print(f7.send(2))
        except Exception as e:
            print(f"{e}")

1
2
6
generator raised StopIteration


StopIteration: 

Пример:

f10 = fact_generator(10)

создает генератор, который при каждом вызове next(f10) возвращает текущее значение факториала, т. е.

next(f10) # вернет 1

next(f10) # вернет 2

next(f10) # вернет 6

...


При этом должен быть реализован функционал, который позволяет послать значение z в генератор и должно вернуться соответствующее значение факториала последовательным применением функции next. Если это невозможно (генератор исчерпался), должно быть выведено сообщение об этом и последнее возможное значение факториала.

Данный функционал должен быть реализован в классе GeneratorManeger:

In [139]:
from functools import wraps, reduce


def coroutine(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        generator = func(*args, **kwargs)
        generator.send(None)  # иначе just-started generator..
        return generator

    return wrapper


@coroutine
def fact_generator(n):
    factor = 0
    res = 1

    while factor <= n:
        upper_limit = yield res
        if upper_limit is not None:
            if upper_limit > factor:
                res *= reduce(lambda a, b: a * b, range(factor + 1, upper_limit + 1))
                factor = upper_limit
            else:
                # попытка запустить исчерпанный генератор
                raise StopIteration(f"Generator rollback, last previous calculated value{factor} {res}")
        else:
            # run without `send` command
            factor += 1
            res *= factor


class HelpGenerator(object):
    def __init__(self, generator, max_n=1):
        self.max_n = max_n or 1
        self.__gen = generator(max_n)
        self.current_n = 0

    def __iter__(self):
        return self

    def __next__(self):
        self.current_n += 1
        return next(self.__gen)

    def __call__(self):
        return self

    def send(self, z):
        return self.__gen.send(z)


class GeneratorManager(object):
    def __init__(self, *fact_desc):
        self.fact_gens = [HelpGenerator(fact_generator, gen_param) for gen_param in fact_desc]

    def send(self, z):
        matched_gen, closed_num = None, 0

        for cur_gen in self.fact_gens:
            if cur_gen.current_n < z <= cur_gen.max_n:
                matched_gen = cur_gen
                break

        if matched_gen is None:
            print(f"Don't have suitable generator {z}!")
            return None

        matched_gen.current_n = z

        return matched_gen.send(z)

fact_desc - это список из чисел $n_i$ для каждого из которых создается свой генератор fact_generator($n_i$) и сохраняется в классе GeneratorManeger.

send должна выбрать любой генератор из еще не исчерпавшихся и вычислить значение факториала. Если таких нет, сообщить об этом.

Пример работы:

In [140]:
gm = GeneratorManager(3, 6)

In [141]:
gm.send(3)

6

In [142]:
gm.send(3)

6

In [143]:
gm.send(3)

Don't have suitable generator 3!


In [144]:
gm.send(5)

120

In [145]:
gm.send(6)

720

In [146]:
gm.send(1)

Don't have suitable generator 1!
